In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import resnet50
from torch.optim import Adam
import torch.nn as nn
import pandas as pd
import os
from PIL import Image
import numpy as np

In [7]:
CELEBA_DATA_PATH = '../'
IMG_PATH = os.path.join(CELEBA_DATA_PATH, 'img_align_celeba')
ATTR_PATH = os.path.join(CELEBA_DATA_PATH, 'list_attr_celeba.csv')
PARTITION_PATH = os.path.join(CELEBA_DATA_PATH, 'partitioned.csv')
merged_path = "../partitioned_multi_attr.csv"

def getImagePath(image_id):
    return os.path.join(IMG_PATH,image_id)

In [ ]:
# Set the data paths
# CELEBA_DATA_PATH = './Data/celeba'
# IMG_PATH = os.path.join(CELEBA_DATA_PATH, 'img_align_celeba/img_align_celeba')
# ATTR_PATH = os.path.join(CELEBA_DATA_PATH, 'list_attr_celeba.csv')
# PARTITION_PATH = os.path.join(CELEBA_DATA_PATH, 'list_eval_partition.csv')
# merged_path = "./Data/celeba/partitioned_multi_attr.csv"





# Load the data|
attributes_df = pd.read_csv(ATTR_PATH)
partitioned_df = pd.read_csv(PARTITION_PATH)

# Calculate and sort the correlations
correlations = attributes_df.drop(columns=['image_id']).corrwith(attributes_df['Male']).abs().sort_values(ascending=False)

# # Select attributes with high correlation and exclude subjective ones
# selected_attributes = correlations[correlations > 0.2].index.difference(['Attractive', 'Chubby', 'High_Cheekbones'])

# # Merge the DataFrames
# merged_df = pd.merge(partitioned_df, attributes_df[['image_id'] + selected_attributes.tolist()], on='image_id')

# # Convert to 0 and 1
# merged_df[merged_df.select_dtypes(include=['number']).columns] = merged_df.select_dtypes(include=['number']).clip(lower=0)

# male_column = merged_df.pop('Male')  # remove Male column and store it
# merged_df.insert(1, 'Male', male_column)  # insert Male column after image_id

# # Export 
# merged_df.to_csv("./partitioned_multi_attr.csv", index=False)

# # Display the first rows of the merged DataFrame
# merged_df.head()

In [8]:
from torch.utils.data import Dataset
from PIL import Image
import os

class CelebADataset(Dataset):
    def __init__(self, file_paths, file_to_label, transform=None):
        self.file_paths = file_paths
        self.file_to_label = file_to_label
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_name = self.file_paths[idx]
        image = Image.open(img_name).convert('RGB')
        label = self.file_to_label[os.path.basename(img_name)][0]  # Only the "Male" label
        attributes = self.file_to_label[os.path.basename(img_name)][2:]  # After partition rest are attributes
        if self.transform:
            image = self.transform(image)

        return image, label, torch.tensor(attributes, dtype=torch.float32)


    
df = pd.read_csv("../partitioned_multi_attr.csv")
train_df = df[df['partition'] == 0]
val_df = df[df['partition'] == 1]
test_df = df[df['partition'] == 2]

df_labels = df.set_index('image_id')
filename_to_label = {filename: labels.values for filename, labels in df_labels.iterrows()}
file_paths = df['image_id'].apply(getImagePath).tolist()


# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [9]:
# Assuming the images are in a directory named 'images' in the current working directory
# Create separate file path and label mappings for each dataset partition
train_file_paths = train_df['image_id'].apply(getImagePath).tolist()
val_file_paths = val_df['image_id'].apply(getImagePath).tolist()
test_file_paths = test_df['image_id'].apply(getImagePath).tolist()

train_filename_to_label = {filename: labels.values for filename, labels in train_df.set_index('image_id').iterrows()}
val_filename_to_label = {filename: labels.values for filename, labels in val_df.set_index('image_id').iterrows()}
test_filename_to_label = {filename: labels.values for filename, labels in test_df.set_index('image_id').iterrows()}

# Initialize the datasets for each partition
train_dataset = CelebADataset(train_file_paths, train_filename_to_label, transform=transform)
val_dataset = CelebADataset(val_file_paths, val_filename_to_label, transform=transform)
test_dataset = CelebADataset(test_file_paths, test_filename_to_label, transform=transform)

# Create data loaders for each dataset partition
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [10]:
# Load a pre-trained ResNet model
model = resnet50(pretrained=True)

# Modify the model for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: Male/Female

C:\Users\hunts\anaconda3\envs\NN_Proj\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hunts\anaconda3\envs\NN_Proj\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
from torchvision.models import resnet50
import torch.nn as nn

# Load a pre-trained ResNet model
model = resnet50(pretrained=True)

# Assume the number of attributes is equal to the number of columns minus 3 (for the image_id, male, partition)
num_attributes = df.shape[1] - 3

# Modify the model for binary classification plus additional attributes
class MultiInputResNet(nn.Module):
    def __init__(self, base_model, num_attributes, num_classes=2):
        super().__init__()
        self.base_model = base_model
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()  # Remove the original fully connected layer

        # Fully connected layer for attributes
        self.attr_fc = nn.Linear(num_attributes, 224)  

        # Final fully connected layer that takes both image features and attributes
        self.final_fc = nn.Linear(num_ftrs + 224, 1)  # Modify num_classes based on your classification problem

    def forward(self, image, attributes):
        # Get image features from the ResNet
        img_features = self.base_model(image)

        # Process attributes
        attr_features = self.attr_fc(attributes)

        # Concatenate image and attribute features
        combined_features = torch.cat((img_features, attr_features), dim=1)

        # Final classification layer
        return self.final_fc(combined_features)

# Update the model
multi_attr_model = MultiInputResNet(base_model=model, num_attributes=num_attributes)


In [17]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=5, verbose=False, delta=0, checkpoint_name="muti_attr_checkpoint.pt"):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            checkpoint_name (str): Name of the checkpoint file. 
                            Default: "checkpoint.pt"
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.checkpoint_name = checkpoint_name 

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decreases."""
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.checkpoint_name) 
        self.val_loss_min = val_loss

        
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import torch

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, checkpoint_name):
    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True, checkpoint_name=checkpoint_name)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_pbar = tqdm(train_loader, unit="batch")

        for images, labels, attributes in train_pbar:
            train_pbar.set_description(f"Epoch {epoch+1}/{num_epochs}")
            images, labels, attributes = images.to(device), labels.to(device).float(), attributes.to(device).float()
            
            optimizer.zero_grad()
            outputs = model(images, attributes)
            
            labels = labels.view(-1, 1) if labels.ndim == 1 else labels
            
            loss = criterion(outputs, labels)  
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
            train_pbar.set_postfix(loss=loss.item())

        train_loss = running_loss / len(train_loader.dataset)

        model.eval()
        val_running_loss = 0.0
        val_pbar = tqdm(val_loader, unit="batch")

        with torch.no_grad():
            for images, labels, attributes in val_pbar:
                val_pbar.set_description(f"Val Epoch {epoch+1}/{num_epochs}")
                images, labels, attributes = images.to(device), labels.to(device).float(), attributes.to(device).float()
                
                outputs = model(images, attributes)
                
                labels = labels.view(-1, 1) if labels.ndim == 1 else labels
                
                loss = criterion(outputs, labels)  
                val_running_loss += loss.item() * images.size(0)
                val_pbar.set_postfix(loss=loss.item())

        val_loss = val_running_loss / len(val_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs} Train loss: {train_loss:.4f} Val loss: {val_loss:.4f}')

        # Call early stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        scheduler.step(val_loss)

    model.load_state_dict(torch.load(checkpoint_name))
    return model


In [21]:
import torch
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from torchvision.models import resnet50
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

# Assuming the `MultiInputResNet` class has been defined as discussed previously

# Check if GPU is available and move the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
multi_attr_model.to(device)

# Assuming train_loader and val_loader have been defined and are ready to use
# Define the loss function and optimizer
criterion = BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1, verbose=True)

# Define the number of epochs
num_epochs = 20  # Change this to your desired number of epochs

# Run the training function
trained_model = train_model(
    model=multi_attr_model, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler, 
    num_epochs=num_epochs, 
    device=device, 
    checkpoint_name="multi_attr_checkpoint.pt"
)

# Save the trained model state
torch.save(trained_model.state_dict(), "multi_attr_model_final.pt")

print("Training complete.")


Val Epoch 1/20: 100%|████████████████████████████████████████████████| 157/157 [00:08<00:00, 17.95batch/s, loss=0.0461]


Epoch 1/20 Train loss: 0.0770 Val loss: 0.0779
Validation loss decreased (inf --> 0.077858).  Saving model ...


Val Epoch 2/20: 100%|████████████████████████████████████████████████| 157/157 [00:08<00:00, 17.74batch/s, loss=0.0276]


Epoch 2/20 Train loss: 0.0634 Val loss: 0.0664
Validation loss decreased (0.077858 --> 0.066408).  Saving model ...


Val Epoch 3/20: 100%|████████████████████████████████████████████████| 157/157 [00:08<00:00, 17.84batch/s, loss=0.0208]


...

Epoch 10/20 Train loss: 0.0020 Val loss: 0.0860


Val Epoch 11/20: 100%|██████████████████████████████████████████████| 157/157 [00:08<00:00, 19.54batch/s, loss=0.00199]


Epoch 11/20 Train loss: 0.0011 Val loss: 0.0975
Epoch 00011: reducing learning rate of group 0 to 1.0000e-05.


Val Epoch 12/20: 100%|██████████████████████████████████████████████| 157/157 [00:08<00:00, 19.58batch/s, loss=0.00142]


Epoch 12/20 Train loss: 0.0007 Val loss: 0.0995
Early stopping
Training complete.


In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def test_model(model, dataloader, device, criterion):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for images,labels, attributes  in tqdm(dataloader, desc="Testing"):
            # Move data to the appropriate device
            images = images.to(device).float() 
            attributes = attributes.to(device).float()  
            labels = labels.to(device).float() 
            # Forward pass to get the logits
            outputs = model(images, attributes).squeeze()

            # Calculate loss
            loss = criterion(outputs, labels.float())
            test_loss += loss.item()

            # Convert logits to probabilities
            probs = torch.sigmoid(outputs).squeeze()

            # Convert probabilities to predicted class (0 or 1)
            predictions = (probs >= 0.5).int()

            # Store true labels and predictions for later evaluation
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

    # Calculate the average loss
    test_loss /= len(dataloader.dataset)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f'Test Loss: {test_loss:.4f}')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

    return test_loss, accuracy, precision, recall, f1



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state_dict = torch.load("multi_attr_model_final.pt", map_location=device)
multi_attr_model.load_state_dict(state_dict)
multi_attr_model.to(device)


# Assuming that the model has been trained and the state_dict has been loaded
# and that test_loader has been defined
test_loss, accuracy, precision, recall, f1 = test_model(
    model=multi_attr_model,
    dataloader=test_loader,
    device=device,
    criterion=criterion
)

Testing: 100%|███████████████████████████████████████████████████████████████████████| 157/157 [00:08<00:00, 18.30it/s]

Test Loss: 0.0020
Accuracy: 0.9808
Precision: 0.9779
Recall: 0.9760
F1 Score: 0.9770
